In [1]:
from gensim import corpora
from gensim.models import LdaModel
import json
from data_analysis.OpeningLDA import OpeningLDA
from flask_.QueryHandler import QueryHandler

In [2]:
#opening_lda = OpeningLDA(num_passes=15) 

#print(len(opening_lda.texts.keys()))
#opening_lda.lda_model.print_topics(num_words=15)

#opening_lda.save('lda1')
opening_lda = OpeningLDA.load('lda1')

8674


In [3]:
query_handler = QueryHandler(load_from_file='lda1')

8674


In [4]:
sample_query1 = "I prefer to play aggressive, castle early, and prevent queen-side attacks"

In [5]:
query_handler.handle_user_query(sample_query1)

['prefer', 'play', 'aggressive', 'castle', 'early', 'prevent', 'queen', 'side', 'attacks']
8674
[{'A10': 1, 'A30': 1, 'A20': 1, 'A43': 1, 'B10': 1, 'B20': 1, 'B21': 1, 'C42': 1, 'D02': 2, 'C43': 1, 'B12': 1, 'B13': 1, 'C55': 1, 'E20': 2, 'A60': 1, 'A57': 1, 'B09': 1, 'E32': 2, 'B36': 1, 'B18': 1, 'D43': 1, 'B44': 1, 'B41': 1, 'E41': 2, 'E24': 2, 'E42': 2, 'B70': 1, 'B90': 1, 'B80': 1, 'B60': 1}, {'A00': 2, 'B00': 6, 'A01': 19, 'A10': 23, 'A02': 15, 'A04': 6, 'A30': 14, 'A80': 8, 'A20': 14, 'B07': 16, 'A43': 4, 'B10': 19, 'C00': 42, 'B02': 22, 'B01': 15, 'B20': 48, 'B22': 6, 'B21': 46, 'C23': 6, 'D00': 13, 'A45': 7, 'A09': 6, 'C31': 21, 'D07': 11, 'C41': 12, 'A53': 3, 'D08': 10, 'A51': 39, 'A82': 5, 'C30': 22, 'C42': 11, 'E60': 30, 'D10': 22, 'D20': 8, 'C33': 21, 'D02': 21, 'D03': 17, 'C02': 36, 'C60': 12, 'C43': 11, 'C21': 9, 'C45': 15, 'C03': 36, 'B12': 15, 'B13': 11, 'C01': 36, 'D01': 5, 'C47': 9, 'C55': 11, 'C65': 2, 'C50': 5, 'E61': 23, 'C15': 36, 'E20': 39, 'D80': 15, 'A60': 6, 'A